In [12]:
#| default_exp preprocessing.lesson15

# Lesson15 Scripts
> Reproducing lesson 15

In [13]:
#| hide
%load_ext autoreload
%autoreload 2

In [14]:
#| export
from cv_tools.core import *


## learner[ 1:16:01]

In [109]:
import torch
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections.abc import Mapping

In [16]:
from datasets import load_dataset, load_dataset_builder

In [17]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)

import logging
logging.disable(logging.WARNING)

In [18]:
name = 'fashion_mnist'
dsd = load_dataset(name)
x, y = 'image', 'label'


In [76]:
from miniai.datasets import *
from operator import itemgetter
from torch.utils.data import default_collate
from torch.utils.data import DataLoader
from fastcore.all import *
from miniai.training import *

In [23]:
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [40]:
@inplace
def transformi(b):
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [41]:
bs = 1024
tsd = dsd.with_transform(transformi)

In [58]:
dsd['train'][0][x].size

(28, 28)

In [45]:
tsd['train']['image'][1].shape


torch.Size([784])

In [ ]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [60]:
tsd

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [61]:
dls = get_dls(tsd['train'], tsd['test'], bs)

In [63]:
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [64]:
class DataLoaders:
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))

In [73]:
from miniai.conv import *

In [74]:
Mapping?

Object `Mapping` not found.


In [77]:
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

def to_device(x, device=def_device):
    if isinstance(x, torch.Tensor): return x.to(device)
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(to_device(o, device) for o in x)

In [106]:
tds = dsd.with_transform(transformi)

In [114]:
class Learner:
	def __init__(self, model, dls, loss_func, opt_func=optim.SGD, lr=1e-1):
		store_attr()

	def one_batch(self):
		self.xb, self.yb = to_device(self.batch)
		self.preds = self.model(self.xb)
		self.loss = self.loss_func(self.preds, self.yb)
		if self.model.training:
			self.loss.backward()
			self.opt.step()
			self.opt.zero_grad()
		with torch.no_grad():
			self.calc_stats()
	
	def calc_stats(self):
		acc = (self.preds.argmax(dim=1).eq(self.yb).float().sum())
		self.accs.append(acc)
		n = len(self.yb)
		self.losses.append(self.loss*n)
		self.ns.append(n)

	def one_epoch(self, train):
		self.model.training = train
		dl = self.dls.train if train else self.dls.valid
		for self.num, self.batch in enumerate(dl):
			self.one_batch()
		n = sum(self.ns)
		#print(f'epoch {self.epoch} , {self.model.training}, {(self.losses).item()/n:.2f} , {sum(self.accs).item()/n:.2f}')
		print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
	

	def fit(self, n_epochs):
		self.accs, self.losses, self.ns = [], [], []

		self.model.to(def_device)
		self.opt = self.opt_func(self.model.parameters(), self.lr)
		self.n_epochs = n_epochs
		for self.epoch in range(n_epochs):
			self.one_epoch(True)
			with torch.no_grad():
				self.one_epoch(False)
	


In [115]:
m, nh =28*28,50
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

learner = Learner(model=model, dls=dls, loss_func=F.cross_entropy, opt_func=optim.SGD, lr=0.2)
learner.fit(1)

0 True 1.15813828125 0.6121333333333333
0 False 1.1153474330357143 0.6226


In [31]:
#| hide
import nbdev; nbdev.nbdev_export('01_preprocessing.lesson_15.ipynb')